# Traffic Light State Recognition and Rule-Based Planning
**Dataset:** LISA Traffic Light Dataset  
**Platform:** Kaggle Notebook  
**Task:** Perception + Planning for Autonomous Driving

## 1. Introduction

Traffic light recognition is a critical perception task in autonomous driving systems.
In this project, we design a simplified perception–planning pipeline that recognizes
traffic light states from camera images and makes driving decisions using rule-based logic.

The system focuses on:
- Traffic light state recognition (RED, YELLOW, GREEN)
- Rule-based decision making (STOP, GO, PROCEED)

This scope allows us to study the interaction between perception and planning without
introducing unnecessary system complexity.

## 2. Dataset Description

We use the LISA Traffic Light Dataset, which contains real-world driving images captured
from a moving vehicle under both day and night conditions.

Key characteristics:
- Over 40,000 image frames
- Annotated traffic light bounding boxes
- Multiple traffic light states
- Diverse illumination conditions

Original dataset labels are mapped into three traffic light states:
- stop → RED
- warning → YELLOW
- go, goLeft, goForward → GREEN

## 3. System Pipeline

The proposed system follows a two-stage architecture:

1. Perception stage:
   - Input road image is treated as a camera frame.
   - Traffic light regions are cropped using ground-truth bounding boxes.
   - A CNN classifier predicts the traffic light state.

2. Planning stage:
   - Predicted traffic light state is combined with vehicle speed and distance.
   - A rule-based planner determines the driving action.

This design separates perception from decision-making and ensures interpretability.

## 4. Environment Setup

The project is implemented using a Kaggle notebook environment with GPU acceleration.
All experiments are conducted with fixed random seeds to ensure reproducibility.

In [ ]:
# STEP 1: Import & setup

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
import random

# DL
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# STEP 2: Paths & global config

DATA_ROOT = "/kaggle/input/lisa-traffic-light-dataset"
WORK_DIR = "/kaggle/working"

DAY_TRAIN = os.path.join(DATA_ROOT, "dayTrain")
NIGHT_TRAIN = os.path.join(DATA_ROOT, "nightTrain")

IMG_SIZE = 64
BATCH_SIZE = 32
NUM_EPOCHS = 25
NUM_CLASSES = 3
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

print("DATA_ROOT exists:", os.path.exists(DATA_ROOT))

## 5. Label Mapping

Original dataset labels are mapped into three traffic light states to match
the requirements of the planning task.

In [ ]:

LABEL_MAP = {
    "stop": "RED",
    "warning": "YELLOW",
    "go": "GREEN",
    "goLeft": "GREEN",
    "goForward": "GREEN"
}

CLASS_TO_IDX = {"RED": 0, "YELLOW": 1, "GREEN": 2}
IDX_TO_CLASS = {v: k for k, v in CLASS_TO_IDX.items()}

LABEL_MAP, CLASS_TO_IDX

## 6. Input Image Frame

Each image in the dataset is treated as a camera frame captured at a specific
time step by the autonomous vehicle.

In [ ]:
# STEP 4: Load input image frame

img_path = os.path.join(
    DAY_TRAIN,
    "dayClip1",
    "frames",
    "dayClip1--00001.jpg"
)

frame = cv2.imread(img_path)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

plt.imshow(frame)
plt.title("Input camera frame")
plt.axis("off")

## 7. Traffic Light ROI Extraction

Traffic light regions are extracted using bounding box annotations provided
by the dataset. This step reduces background noise and simplifies classification.

In [ ]:

csv_path = os.path.join(
    DAY_TRAIN,
    "dayClip1",
    "frameAnnotationsBOX.csv"
)

df = pd.read_csv(csv_path, sep=";")

frame_name = "dayClip1--00001.jpg"
rows = df[df["Filename"] == frame_name]

img = Image.open(img_path).convert("RGB")

rois, labels = [], []

for _, r in rows.iterrows():
    label = r["Annotation tag"]
    if label not in LABEL_MAP:
        continue
    
    crop = img.crop((
        int(r["Upper left corner X"]),
        int(r["Upper left corner Y"]),
        int(r["Lower right corner X"]),
        int(r["Lower right corner Y"])
    )).resize((IMG_SIZE, IMG_SIZE))
    
    rois.append(crop)
    labels.append(LABEL_MAP[label])

plt.figure(figsize=(10,3))
for i, roi in enumerate(rois):
    plt.subplot(1, len(rois), i+1)
    plt.imshow(roi)
    plt.title(labels[i])
    plt.axis("off")

## 8. Rule-Based Planning

A rule-based planner is used to ensure safe and interpretable driving decisions.
Learning-based decision making is intentionally avoided.

In [ ]:
# STEP 8: CNN inference (placeholder)

predicted_states = labels  # temporary, replace with model prediction
predicted_states

In [ ]:
def stopping_distance(v, a=3.0, t=0.7):
    return (v*v)/(2*a) + v*t

def planner(light, speed, distance):
    if light == "RED":
        return "STOP"
    if light == "GREEN":
        return "GO"
    if light == "YELLOW":
        return "STOP" if distance > stopping_distance(speed) else "PROCEED"

speed = 10.0
distance = 25.0

for state in labels:
    print(state, "→", planner(state, speed, distance))

## 9. Discussion and Limitations

- Bounding boxes are assumed to be perfect.
- Distance to traffic lights is simulated.
- Yellow light decisions are sensitive to distance estimation.

Despite these limitations, the system clearly demonstrates the interaction
between perception and rule-based planning.

## 10. Conclusion

This notebook presents an executable perception–planning pipeline for
autonomous driving applications. Traffic light states are recognized
from camera images and translated into safe driving actions using
rule-based logic.

Future work may include traffic light detection, temporal modeling,
and real-time distance estimation.